### Copy and compress video files

In [1]:
from pathlib2 import Path
import pathlib2 
import os

In [2]:
from tqdm import tqdm

In [3]:
# settings
crf = 15 # 0 = lossless, 51 = worst

In [4]:
# mirror flip bottom cameras 
cameras_to_mirror = []#["FB", "BB"]

In [18]:
base_path_local = Path("/media/dattalab/starling_rec/recordings/")
base_path_save= Path('/n/groups/datta/tim_sainburg/datasets/3D_keypoints/compressed/') 
subfolder = '23-08-14-chronic_recordings'

In [19]:
# '/home/dattalab/JARVIS/sync_flir_azure_9_27/recordings/Recording_2022-09-27T122406'
# folder with recordings
video_folder = csv_folder = base_path_local / subfolder
# folder to save recordings folder
output_folder = base_path_save / subfolder

In [20]:
if not os.path.exists('/n/groups/datta/tim_sainburg'):
    raise ValueError('must mount /n/groups/datta')

In [21]:
save_folder = output_folder /video_folder.stem

In [22]:
def ensure_dir(file_path):
    """ create a safely nested folder
    """
    if type(file_path) == str:
        if "." in os.path.basename(os.path.normpath(file_path)):
            directory = os.path.dirname(file_path)
        else:
            directory = os.path.normpath(file_path)
        if not os.path.exists(directory):
            try:
                os.makedirs(directory)
            except FileExistsError as e:
                # multiprocessing can cause directory creation problems
                print(e)
    elif type(file_path) == pathlib2.PosixPath:
        # if this is a file
        if len(file_path.suffix) > 0:
            file_path.parent.mkdir(parents=True, exist_ok=True)
        else:
            file_path.mkdir(parents=True, exist_ok=True)

In [23]:
ensure_dir(save_folder)

In [24]:
videos_to_convert = list(video_folder.glob('**/*.avi'))

In [25]:
print(len(videos_to_convert))

15


In [26]:
list(videos_to_convert)

[PosixPath('/media/dattalab/starling_rec/recordings/23-08-14-chronic_recordings/23-02-15-13-33-48-456374/Top.22181547.avi'),
 PosixPath('/media/dattalab/starling_rec/recordings/23-08-14-chronic_recordings/23-02-15-13-33-48-456374/BackLeft.22181546.avi'),
 PosixPath('/media/dattalab/starling_rec/recordings/23-08-14-chronic_recordings/23-02-15-13-33-48-456374/FrontLeft.22181612.avi'),
 PosixPath('/media/dattalab/starling_rec/recordings/23-08-14-chronic_recordings/23-02-15-13-33-48-456374/FrontRight.22181541.avi'),
 PosixPath('/media/dattalab/starling_rec/recordings/23-08-14-chronic_recordings/23-02-15-13-33-48-456374/BackRight.22181539.avi'),
 PosixPath('/media/dattalab/starling_rec/recordings/23-08-14-chronic_recordings/23-02-15-13-03-25-815191/Top.22181547.avi'),
 PosixPath('/media/dattalab/starling_rec/recordings/23-08-14-chronic_recordings/23-02-15-13-03-25-815191/BackLeft.22181546.avi'),
 PosixPath('/media/dattalab/starling_rec/recordings/23-08-14-chronic_recordings/23-02-15-13-03-2

In [27]:
import ffmpeg
import numpy as np
from tqdm.auto import tqdm
import subprocess

In [28]:
#ffmpeg -i FL.avi -c:v libx265 -vtag hvc1 -crf 20 -preset ultrafast FL_compressed-265-2.avi

In [29]:
from IPython.display import clear_output
import os
import shutil

In [30]:
delete_local_video = False

In [31]:
# copy all metadata (metadata.csv and triggerdata.csv)
metadata_to_copy = list(csv_folder.glob('**/metadata.csv'))
triggerdata_to_copy = list(csv_folder.glob('**/triggerdata.csv'))
for csv_file in tqdm(metadata_to_copy + triggerdata_to_copy):
    # get the name to save for the output folder
    number_of_parents_to_base = np.where([i == csv_folder for i in list(csv_file.parents)])[0][0]
    sub_folders = csv_file.parts[len(csv_file.parts)-1-number_of_parents_to_base:-1]
    csv_out = output_folder
    for i in sub_folders:
        csv_out = csv_out / i
    csv_out = csv_out / csv_file.name
    
    ensure_dir(csv_out)
    
    if csv_out.exists():
        print('Video already exists, skipping \n\t {}'.format('/'.join(video.parts[-3:])))

        #if delete_local_video:
            #os.remove(csv_file)

        continue
        
    else:
        shutil.copyfile(csv_file, csv_out)

  0%|          | 0/3 [00:00<?, ?it/s]

In [32]:
for video in tqdm(videos_to_convert):
    try:
        # get the name to save for the output folder
        number_of_parents_to_base = np.where([i == video_folder for i in list(video.parents)])[0][0]
        sub_folders = video.parts[len(video.parts)-1-number_of_parents_to_base:-1]
        video_out = output_folder
        for i in sub_folders:
            video_out = video_out / i
        video_out = video_out / video.name

        ensure_dir(video_out)

        if video_out.exists():
            print('Video already exists, skipping \n\t {}'.format('/'.join(video.parts[-3:])))
            
            if delete_local_video:
                os.remove(video)
            
            continue

        print('converting {}'.format('/'.join(video.parts[-3:])))
        #continue
        # https://www.programcreek.com/python/example/117479/ffmpeg.output

        flip_video = video.stem in cameras_to_mirror

        ffmpeg_command = (
            ffmpeg
            .input(video.as_posix())

        )
        if flip_video:
            ffmpeg_command = ffmpeg_command.video.hflip()
        ffmpeg_command = (
            ffmpeg_command.output(
                video_out.as_posix(),
                vcodec = "libx265", 
                crf = crf, 
                vtag="hvc1", 
                preset="ultrafast"
            )
        )
        out, _ = ffmpeg_command.run()
        
        
        # if out is successful 
        if delete_local_video:
            os.remove(video)
        
        #out, _ = (
        #    ffmpeg
        #    .input(video.as_posix())
        #    .output(video_out.as_posix(), vcodec = "libx265", crf = crf, vtag="hvc1", preset="ultrafast")
        #    .run()
        #)
    
        
    except KeyError:
        # delete bad video file
        if video_out.exists():
            os.remove(video_out)
        break
        
    except Exception:
        # delete bad video file
        if video_out.exists():
            os.remove(video_out)
        
        clear_output(wait=True)
        print('BACKUP FAILED {}'.format(video))
        continue

  0%|          | 0/15 [00:00<?, ?it/s]

converting 23-08-14-chronic_recordings/23-02-15-13-33-48-456374/Top.22181547.avi


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

converting 23-08-14-chronic_recordings/23-02-15-13-33-48-456374/BackLeft.22181546.avi


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

converting 23-08-14-chronic_recordings/23-02-15-13-33-48-456374/FrontLeft.22181612.avi


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

KeyboardInterrupt: 